In [ ]:
import bs4
import requests
import pprint
import pymysql, warnings
import time

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [ ]:
driver = webdriver.Chrome(executable_path='./webdriver/chromedriver')
driver.get('https://www.vivareal.com.br/venda/santa-catarina/itapema/')
htmlPage = bs4.BeautifulSoup(driver.page_source,'html.parser')
htmlElem = driver.find_element_by_tag_name('html')

In [ ]:
content = htmlPage.find_all('div',{'class':'js-card-selector'})
price = content[0].find('div',{'class':'property-card__price js-property-card-prices js-property-card__price-small'}).text

In [ ]:
def scroll():
    htmlElem.send_keys(Keys.HOME)
    tente = True
    count = 0
    while tente:
        try:
            htmlElem.send_keys(Keys.END)
            time.sleep(3)
            htmlElem.send_keys(Keys.PAGE_UP)
            time.sleep(3)
            htmlElem.send_keys(Keys.PAGE_UP)
            count +=1
            time.sleep(1)
            driver.find_elements_by_xpath('//a[@title="Próxima página"]')[0].click()
            time.sleep(3)
            htmlPage = bs4.BeautifulSoup(driver.page_source,'html.parser')
            content = htmlPage.find_all('div',{'class':'js-card-selector'})
            new_price = content[0].find('div',{'class':'property-card__price js-property-card-prices js-property-card__price-small'}).text
            
            if new_price == price:
                tente = True
            else:
                return True
            
        except BaseException as error:
            global e
            e = error
            pass

In [ ]:
def getImoveisInfo():
    content = htmlPage.find_all('div',{'class':'js-card-selector'})
    global imoveis
    imoveis = []
    for imovel in content:
        #dicionario que ira receber todas as informacoes
        imoveis_info = {'preco':'', 'endereco':'','tamanho':'','quartos':'','banheiros':'','descricao':'','imagem':'','name':'','url':'', 'vagas_garagem':''}

        imoveis_info['preco'] = imovel.find('div',{'class':'property-card__price js-property-card-prices js-property-card__price-small'}).text
        imoveis_info['endereco'] = imovel.find('span',{'class':'property-card__address js-property-card-address js-see-on-map'}).text
        imoveis_info['tamanho'] = imovel.find('li',{'class':'property-card__detail-item property-card__detail-area'}).find('span').text
        imoveis_info['quartos'] = imovel.find('li',{'class':'property-card__detail-item property-card__detail-room js-property-detail-rooms'}).find('span').text
        imoveis_info['banheiros'] = imovel.find('li',{'class':'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'}).find('span').text
        imoveis_info['descricao'] = imovel.find('div',{'class':'property-card__description js-property-description'}).text
        imoveis_info['imagem'] = imovel.find('img',{'class':'carousel__image js-carousel-image'}).get('src')
        imoveis_info['name'] = imovel.find('a',{'class':'property-card__title js-cardLink js-card-title'}).text
        imoveis_info['url'] = imovel.find('a',{'class':'property-card__title js-cardLink js-card-title'}).get('href')
        imoveis_info['vagas_garagem'] = imovel.find('li',{'class':'property-card__detail-item property-card__detail-garage js-property-detail-garages'}).find('span').text

        #cria-se uma lista de dicionarios para todas as informacoes sobre todos os imoveis
        imoveis.append(imoveis_info)
    return imoveis

In [ ]:
def saveImoveisInfo(imoveis_list_info):
    db = pymysql.connect(host='br938.hostgator.com.br', user='blockch1', passwd='ulh46hBT28', db='blockch1_scrappingData')
    for imovel in imoveis_list_info:   
        cursor = db.cursor()
        warnings.simplefilter("ignore")
        query = 'insert into imoveis_itapema(preco, endereco, tamanho, quartos, banheiros, descricao, imagem, name, url, vagas_garagem) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        cursor.execute(query, (imovel['preco'],imovel['endereco'],imovel['tamanho'],imovel['quartos'],imovel['banheiros'],imovel['descricao'], imovel['imagem'], imovel['name'], imovel['url'], imovel['vagas_garagem']))
        db.commit()
    cursor.close()

In [ ]:
saveImoveisInfo(getImoveisInfo())

In [ ]:
contador = 0
while scroll():
    contador +=1
    saveImoveisInfo(getImoveisInfo())
    print('{} - informacoes salvas'.format(contador))
    time.sleep(1)

necessario rodar somente a funcao que salva os dados para a primeira pagina entao clicar no segundo link, o resto faz sozinho